In [ ]:
# Script for creating and training model, run on google colabs for fastest results

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import PIL
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

img_height = 128
img_width = 128
batch_size = 32

In [ ]:
#connect your google drive to the notebook on google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# numpy file of correct name needs to be uploaded in your google drive. change path as required
# the numpy file contains all the data for training
# use the create_training_data_array.py file to create this npy file
td_array = np.load('drive/MyDrive/lego_project/td_array_7cat.npy', allow_pickle=True)


In [ ]:
# checking if GPU is enabled ingoogle collabs
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# functions for increasing contrast for images, using the second function at the moment as it has better results

def increase_contrast_little(s):
    npImage = s

    min=np.min(npImage)        # result=144
    max=np.max(npImage)        # result=216

    # Make a LUT (Look-Up Table) to translate image values
    LUT=np.zeros(256,dtype=np.uint8)
    LUT[min:max+1]=np.linspace(start=0,stop=255,num=(max-min)+1,endpoint=True,dtype=np.uint8)
    s_new = LUT[npImage]
    return s_new

def increase_contrast_more(s):
    minval = np.percentile(s, 2)
    maxval = np.percentile(s, 98)
    npImage = np.clip(s, minval, maxval)

    npImage = npImage.astype(int)

    min=np.min(npImage)        # result=144
    max=np.max(npImage)        # result=216

    # Make a LUT (Look-Up Table) to translate image values
    LUT=np.zeros(256,dtype=np.uint8)
    LUT[min:max+1]=np.linspace(start=0,stop=255,num=(max-min)+1,endpoint=True,dtype=np.uint8)
    s_clipped = LUT[npImage]
    return s_clipped

In [ ]:
# Convert the training data into list and randomize the order to get a fair split for testing and training data
training_data = td_array.tolist()
random.shuffle(training_data)

In [ ]:
# Create x and y lists for the images and its labels (i.e integers from 0 - 6) respectively
x = []
y = []  

for piece, label in training_data:
    x.append(piece)
    y.append(label)

In [ ]:
x = np.array(list(map(increase_contrast_more, x))) #increase contrast of images
x = np.array(x).reshape(-1,128,128,1) #reshape images for the model
y = np.asarray(y)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2) # split the data into testing and training sets

In [ ]:

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal",
                        input_shape=(img_height,
                                    img_width,
                                    1)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
    ])

model = keras.Sequential(
    [
        data_augmentation,
    
        layers.Rescaling(1./255, input_shape = (img_height,img_width,1)), #normalize the data input

        layers.Conv2D(128, 3, padding="same", activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),

        layers.Conv2D(64, 3, padding="same", activation='relu'), #should this be 16 or 32 units? try with more data
        layers.MaxPooling2D(pool_size=(2,2)),

        layers.Conv2D(32, 3, padding="same", activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        
        layers.Conv2D(16, 3, padding="same", activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        
        layers.Dropout(0.1),
        layers.Flatten(),
        layers.Dense(10,activation = 'relu'),
        layers.Dense(7,activation='softmax'), # number of output classes
        # softmax activation on the last layer will output a probability distribution over the output classes. The sum 
        # of all the probabilities will be equal to 1
        
    ]
)        



model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=False),],
    metrics=["accuracy"],
)
# epochs = 25
#model_history = 

# if you don't need the training graphs, can just run model.fit(...)
# model.fit(x_train, y_train, epochs=200, verbose=2, validation_data=(x_test,y_test), batch_size=25)  #i think 25/32 is the best batch size 

# run this to get graphs of the training progress
model_history = model.fit(x_train, y_train, epochs=200, verbose=2, validation_data=(x_test,y_test), batch_size=25)  #i think 25/32 is the best batch size 


Epoch 1/200
191/191 - 10s - loss: 1.8996 - accuracy: 0.1758 - val_loss: 1.8314 - val_accuracy: 0.1629 - 10s/epoch - 54ms/step
Epoch 2/200
191/191 - 6s - loss: 1.8118 - accuracy: 0.2264 - val_loss: 1.8302 - val_accuracy: 0.1654 - 6s/epoch - 30ms/step
Epoch 3/200
191/191 - 6s - loss: 1.7469 - accuracy: 0.2734 - val_loss: 1.7680 - val_accuracy: 0.2880 - 6s/epoch - 30ms/step
Epoch 4/200
191/191 - 6s - loss: 1.7202 - accuracy: 0.3118 - val_loss: 1.7892 - val_accuracy: 0.2175 - 6s/epoch - 30ms/step
Epoch 5/200
191/191 - 6s - loss: 1.6891 - accuracy: 0.3131 - val_loss: 1.6841 - val_accuracy: 0.3317 - 6s/epoch - 30ms/step
Epoch 6/200
191/191 - 6s - loss: 1.6595 - accuracy: 0.3284 - val_loss: 1.6486 - val_accuracy: 0.3174 - 6s/epoch - 29ms/step
Epoch 7/200
191/191 - 6s - loss: 1.6298 - accuracy: 0.3289 - val_loss: 1.7759 - val_accuracy: 0.2586 - 6s/epoch - 29ms/step
Epoch 8/200
191/191 - 6s - loss: 1.6129 - accuracy: 0.3312 - val_loss: 1.6146 - val_accuracy: 0.3350 - 6s/epoch - 30ms/step
Epoch 

In [ ]:
# to get confusion matrix for model with test data
# By definition a confusion matrix C is such that C(i,j) is equal to the number of observations known to be in group i and predicted to be in group j
# columns are predictions, rows are actual labels

prediction = model.predict(x_test)
classes_x=np.argmax(prediction,axis=1)
cm = confusion_matrix(y_test, classes_x)
print(cm)
#print(prediction)

[[176   1   8   0   1   5   1]
 [  6 212   0   1   0   9   2]
 [ 11   0 181   0   1   0   0]
 [  0   1   0 165   1   0   0]
 [  0   0   2   1 105   0   0]
 [  5   8   0   0   0 193   0]
 [  2   9   0   1   1   1  81]]


In [ ]:
# Save the entire model as a SavedModel.
model.save('drive/MyDrive/lego_project/saved_model/final_model')  
# make sure that cd is at C:/Users/amosk/Python then run the program. This will 
# create a SavedModel named my_model in the saved_model folder inside RPI3_project folder

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('drive/MyDrive/lego_project/saved_model/final_model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('drive/MyDrive/lego_project/model.tflite', 'wb') as f:
  f.write(tflite_model)  #save tfllite model as model.tflite


In [ ]:
# to load model
model_loaded = tf.keras.models.load_model('drive/MyDrive/lego_project/saved_model/my_model')